In [1]:
#Importing required dependencies

import os
import numpy as np
import random
import cv2
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import mrcnn.config
from keras.layers import Layer
from mrcnn import utils, visualize
from mrcnn.model import MaskRCNN
from pathlib import Path

In [ ]:
class MaskRCNNConfig(mrcnn.config.Config):
    NAME = "coco_pretrained_model_config"
    IMAGES_PER_GPU = 1
    GPU_COUNT = 1
    NUM_CLASSES = 1 + 80
    DETECTION_MIN_CONFIDENCE = 0.6
    

def getCarBoxes(boxes,class_ids):
    car_boxes = []
    
    for i,box in enumerate(boxes):
        if class_ids[i] in [3,8]:
            car_boxes.append(boxes)
            
    return np.array(car_boxes)

In [ ]:
# Setting the Root directory of the project
ROOT_DIR = "."

MODEL_DIR = os.path.join(ROOT_DIR,"logs")

COCO_MODEL_PATH = os.path.join(ROOT_DIR,"mask_rcnn_coco.h5")

In [ ]:
# Directory of images to run detection on
IMAGE_DIR = "parking_images"


# Create a Mask R-CNN model in inference mode
model = MaskRCNN(mode='inference', config=MaskRCNNConfig(),model_dir=MODEL_DIR)

%time model.load_weights(COCO_MODEL_PATH,by_name=True)


For each object detected in the image fed as input, the model classifies it and returns class ID, which is an integer value identifying each class.

In [ ]:
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [ ]:
# Location of parking spaces (ARRAY)
parked_car_boxes = None

In [ ]:
# Load a random image from images folder
file_names = next(os.walk(IMAGE_DIR))[2]         

In [ ]:
#Printing file names
file_names

In [ ]:
image1 = skimage.io.imread(os.path.join(IMAGE_DIR,file_names[0]))
image2 = skimage.io.imread(os.path.join(IMAGE_DIR,file_names[1]))


In [ ]:
%time result1 = model.detect([image1])    

In [ ]:
%time result2 = model.detect([image2])    

In [ ]:
r1 = result1[0]
r2 = result2[0]


In [ ]:
# Identifying classes in frame
visualize.display_instances(image1,r1['rois'],r1['masks'], r1['class_ids'], class_names, r1['scores'])

In [ ]:
visualize.display_instances(image2,r2['rois'], r2['masks'], r2['class_ids'], class_names, r2['scores'])

In [ ]:
# Filter the results to only get detected car bounding boxes

car_boxes1 = getCarBoxes(r1['rois'],r1['class_ids'])
car_boxes2 = getCarBoxes(r2['rois'],r2['class_ids'])

In [ ]:
car_boxes1[0].shape

In [ ]:
parking_spaces = car_boxes1[0]

# computinig center locations of each spot
center_locs = []

for spot_coords in parking_spaces:
    center_locs.append([int((spot_coords[1]+spot_coords[3])/2), int((spot_coords[0]+spot_coords[2])/2)])


centers = np.array(center_locs)
    

In [ ]:
centers.shape

In [ ]:
# Draw each box on the frame

for i,box in enumerate(parking_spaces):
    y1, x1, y2, x2 = box
    cv2.rectangle(image1, (x1, y1), (x2, y2),(0,0,255),2)
    cv2.circle(image1,(centers[i][0],centers[i][1]),2,(0,0,255),2)

In [ ]:
# show the image on screen

cv2.imshow('image', image1)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
parking_spaces.shape

In [ ]:
car_boxes2.shape

### <a id='section_5'>Detecting availabe spaces</a>



In [ ]:
# How much car overlaps with the bounding boxes of parking spaces

overlaps = mrcnn.utils.compute_overlaps(car_boxes2[0],parking_spaces)

In [ ]:
overlaps.shape

In [ ]:
overlap_prob = overlaps.sum(axis=0)

In [ ]:
for i,box in enumerate(parking_spaces):
    y1, x1, y2, x2 = box
    
    if overlap_prob[i] >= 0.5:
        occupancy_status = (0,0,255)
    
    else:
        occupancy_status = (0,255,0)
    cv2.rectangle(image2,(x1,y1), (x2,y2) , occupancy_status ,1)
    cv2.circle(image2,(centers[i][0],centers[i][1]),2,occupancy_status,2)
        
    

In [ ]:
cv2.imshow('Free Space Detection', image2)
cv2.waitKey(0)
cv2.destroyAllWindows()